In [3]:
import os
import glob
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr  # Correct import for Spearman correlation
from lightning import pytorch as pl
from chemprop import data, featurizers, models, nn

def find_dataset_path(dataset_name):
    current_directory = os.getcwd()
    base_directory = os.path.abspath(os.path.join(current_directory, ".."))
    
    dataset_pattern = os.path.join(base_directory, "**", f"{dataset_name}.csv")
    matches = glob.glob(dataset_pattern, recursive=True)
    
    if matches:
        return matches[0]
    else:
        return None

# Get user input for dataset and columns
dataset_name = input("Please enter the dataset name (e.g., 'Caco2'): ")
smiles_column = input("Please enter column with structures: ")
target_column = input("Please enter target column: ")
target_columns = [target_column]

# Find the dataset path based on the user input
input_path = find_dataset_path(dataset_name)

if input_path is None:
    print(f"Dataset '{dataset_name}' not found.")
else:
    print(f"Dataset found at: {input_path}")

    # Configuration for model training and evaluation
    num_folds = 10
    num_workers = 0

    # Load dataset
    df_input = pd.read_csv(input_path)

    smis = df_input.loc[:, smiles_column].values
    ys = df_input.loc[:, target_columns].values

    # Create MoleculeDatapoints for all SMILES and target values
    all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    metrics = []

    # Train and evaluate across k-fold splits
    for train_index, test_index in kf.split(all_data):
        train_data = [all_data[i] for i in train_index]
        test_data = [all_data[i] for i in test_index]

        featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
        train_dset = data.MoleculeDataset(train_data, featurizer)
        test_dset = data.MoleculeDataset(test_data, featurizer)

        train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
        test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

        mp = nn.BondMessagePassing()
        agg = nn.MeanAggregation()

        scaler = train_dset.normalize_targets()
        output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
        ffn = nn.RegressionFFN(output_transform=output_transform)
        metric_list = [nn.metrics.RMSEMetric(), nn.metrics.MAEMetric()]
        batch_norm = True

        mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

        trainer = pl.Trainer(
            logger=True,
            enable_checkpointing=True,
            enable_progress_bar=True,
            accelerator="cpu",
            devices=1,
            max_epochs=50,
        )

        # Train the model
        trainer.fit(mpnn, train_loader)

        # Make predictions on the test data
        predictions = trainer.predict(mpnn, test_loader)
        predictions = np.concatenate(predictions)
        true_labels = ys[test_index]

        # Calculate regression metrics
        mae = mean_absolute_error(true_labels, predictions)
        rmse = np.sqrt(mean_squared_error(true_labels, predictions))
        r2 = r2_score(true_labels, predictions)
        spearman_corr, _ = spearmanr(true_labels, predictions)

        metrics.append({
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'Spearman': spearman_corr,
        })

    # Convert the collected metrics to a DataFrame and calculate averages
    metrics_df = pd.DataFrame(metrics)
    average_metrics = metrics_df.mean().to_frame(name='Average').T

    # Output the average metrics
    print(average_metrics)

Please enter the dataset name (e.g., 'CYP1A2-Substrate'):  ppbr
Please enter column with structures:  Drug
Please enter target column:  Y


Dataset found at: /Users/aleksashka/admetica/distribution/ppbr/ppbr.csv


/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'output_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['output_transform'])`.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. C

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'output_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['output_transform'])`.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. C

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=50` reached.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |                                                                                                 …

              MAE       RMSE        R2  Spearman
Average  6.945025  11.336832  0.600208  0.768495
